In [1]:
import sys
import os
from os.path import join
import cv2
import torch
import numpy as np
import pandas as pd
import pickle as pkl
from PIL import Image
from tqdm.auto import tqdm, trange
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.append("/n/home12/binxuwang/Github/DiffusionObjectRelation/utils")
from cv2_eval_utils import find_classify_object_masks

positive_threshold = 180 

In [2]:
saveroot = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/results/objrel_rndembdposemb_DiT_B_pilot/latent_store"
figdir = '/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/results/objrel_rndembdposemb_DiT_B_pilot/Figure_latent_feature_heatmap'
syndir = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/results/objrel_rndembdposemb_DiT_B_pilot/synopsis"
os.makedirs(syndir, exist_ok=True)

In [3]:
syn_col = []
feature_names = [
    "obj_vs_others",
    "triangle_vs_others", 
    "circle_vs_others",
    "square_vs_others", 
    "blue_obj_vs_others",
    "red_obj_vs_others",
    "red_circle_vs_others",
    "red_square_vs_others",
    "red_triangle_vs_others", 
    "blue_circle_vs_others", 
    "blue_square_vs_others",
    "blue_triangle_vs_others",
    "topobj_vs_others", 
    "bottomobj_vs_others",
    "topobj_vs_bottomobj"
]
# annot_label = "triangle_vs_others"
for layer_index in [0, 2, 4, 5, 7, 8, 9, 11]:
    for annot_label in feature_names:
        for t_index in reversed(range(14)):
            for training_pass in [("cond",), ("uncond",), ("cond", "uncond")]:
                training_pass_str = "-".join(training_pass)+"Pass"
                if not os.path.exists(join(figdir, f"red_blue_8_relation_diff_layer{layer_index}_t{t_index}_{annot_label}_classifier_{training_pass_str}.pkl")):
                    print(f"No classifier found for at layer {layer_index} t_index {t_index} {annot_label} and training pass {training_pass_str}")
                    continue
                data = pkl.load(open(join(figdir, f"red_blue_8_relation_diff_layer{layer_index}_t{t_index}_{annot_label}_classifier_{training_pass_str}.pkl"), 'rb'))
                clf, boundary_vector, eval_dict = data["classifier"], data["boundary_vector"], data["eval_dict"]
                stats_dict = eval_dict.copy()
                stats_dict["layer_index"] = layer_index
                stats_dict["t_index"] = t_index
                stats_dict["training_pass"] = training_pass_str
                stats_dict["annot_label"] = annot_label
                stats_dict["boundary_vector"] = boundary_vector
                # stats_dict.pop('classifier')
                # stats_dict.pop('boundary_vector')
                syn_col.append(stats_dict)
            # print(f"t_index: {t_index}, training_pass: {training_pass_str}, eval_dict: {eval_dict}")
            # print(f"boundary_vector: {boundary_vector}")
syn_df = pd.DataFrame(syn_col)
# syn_df.to_csv(join(figdir, f"red_blue_8_relation_diff_stats_synopsis.csv"), index=False)
syn_df.to_pickle(join(syndir, f"red_blue_8_relation_feature_classifier_alllayers_stats_synopsis.pkl"))


No classifier found for at layer 0 t_index 10 obj_vs_others and training pass condPass
No classifier found for at layer 0 t_index 10 obj_vs_others and training pass uncondPass
No classifier found for at layer 0 t_index 10 obj_vs_others and training pass cond-uncondPass
No classifier found for at layer 0 t_index 4 obj_vs_others and training pass condPass
No classifier found for at layer 0 t_index 4 obj_vs_others and training pass uncondPass
No classifier found for at layer 0 t_index 4 obj_vs_others and training pass cond-uncondPass
No classifier found for at layer 0 t_index 1 obj_vs_others and training pass condPass
No classifier found for at layer 0 t_index 1 obj_vs_others and training pass uncondPass
No classifier found for at layer 0 t_index 1 obj_vs_others and training pass cond-uncondPass
No classifier found for at layer 0 t_index 0 obj_vs_others and training pass condPass
No classifier found for at layer 0 t_index 0 obj_vs_others and training pass uncondPass
No classifier found fo

In [4]:
syn_col = []
feature_names = [
    "obj_vs_others",
    "triangle_vs_others", 
    "circle_vs_others",
    "square_vs_others", 
    "blue_obj_vs_others",
    "red_obj_vs_others",
    "red_circle_vs_others",
    "red_square_vs_others",
    "red_triangle_vs_others", 
    "blue_circle_vs_others", 
    "blue_square_vs_others",
    "blue_triangle_vs_others",
    "topobj_vs_others", 
    "bottomobj_vs_others",
    "topobj_vs_bottomobj"
]
# annot_label = "triangle_vs_others"
missing_collection = []
for layer_index in [0, 2, 4, 5, 7, 8, 9, 11]:
    for annot_label in feature_names:
        for t_index in reversed(range(14)):
            for training_pass in [("cond",), ("uncond",), ("cond", "uncond")]:
                training_pass_str = "-".join(training_pass)+"Pass"
                if not os.path.exists(join(figdir, f"red_blue_8_relation_diff_layer{layer_index}_t{t_index}_{annot_label}_classifier_{training_pass_str}.pkl")):
                    # print(f"No classifier found for at layer {layer_index} t_index {t_index} {annot_label} and training pass {training_pass_str}")
                    missing_collection.append({"layer_index": layer_index, "t_index": t_index, "annot_label": annot_label, "training_pass": training_pass_str})
                else:
                    continue
missing_df = pd.DataFrame(missing_collection)
# missing_df.to_csv(join(syndir, f"red_blue_8_relation_feature_classifier_missing_stats_synopsis.csv"), index=False)

In [10]:
missing_df.groupby(['layer_index', 't_index']).count()

annot_label  training_pass
layer_index t_index                            
0           0                 45             45
            1                 45             45
            2                 23             23
            4                 45             45
            5                  3              3
            7                 42             42
            10                45             45
            11                40             40
2           0                 45             45
            1                 31             31
            4                 34             34
            10                27             27
4           0                 44             44
            4                  4              4
            7                 24             24
            10                45             45
            11                 8              8
5           0                 45             45
            1                 16             16
            4                 20             20
            7                  1              1
            10                28             28
7           0                 45             45
            1                  6              6
            10                 3              3
8           0                 45             45
            1                 12             12
            4                 13             13
            7                 29             29
            10                45             45
            11                15             15
9           0                 45             45
            1                 13             13
            4                 13             13
            7                  6              6
            10                37             37
11          0                 45             45
            1                 27             27
            4                 24             24
            7                 45             45
            8                  1              1
            10                45             45
            11                45             45
            12                13             13

In [9]:
missing_df[['layer_index', 't_index']].drop_duplicates().sort_values(by=['layer_index', 't_index'])

,layer_index,t_index
9,0,0
6,0,1
78,0,2
3,0,4
200,0,5
18,0,7
0,0,10
12,0,11
288,2,0
294,2,1
